In [13]:
#   載入套件
from bs4 import BeautifulSoup as bs
import requests
import re
import pandas as pd


In [14]:
#   建立函式
def get_ptt_post(soup):

    #   取得文章列表
    #   目標 div
    data = soup.select("div.r-ent")
    result = []
    for i in data:

        #   標題
        title = i.select("div.title")[0].text.strip()

        #   忽略 公告類文章 & 已刪文文章
        if "公告" in title or ("已被" in title and "刪除" in title):
            continue

        #   發文時間
        date = i.select("div.date")[0].text.strip()

        #   作者
        author = i.select("div.author")[0].text.strip()

        #   網址
        #   PTT 網站
        oriLink = "https://www.ptt.cc" + i.select("div.title a")[0]["href"]

        #   取得文章相關資訊
        #   內文
        #   請求文章內文
        res_content = requests.get(oriLink)
        soup_content = bs(res_content.text,"lxml")
        #  文章內容簡易驗證
        results_content = soup_content.select('span.article-meta-value')
        if len(results_content) > 3:
            #   驗證成功, 篩出文章內文
            content = soup_content.find(id="main-content").text
            Footer = u'※ 發信站: 批踢踢實業坊(ptt.cc),'
            #   移除註腳以下內容
            content = content.split(Footer)
            #   存取內容
            main_content = content[0]
            pass
        else:
            #   驗證失敗, 跳過該文章
            print(oriLink,"內文異常:ID/版標/標題/日期為空")
            print(results_content)
            continue

        result.append({
            "title":title,
            "date":date,
            "author":author,
            "link":oriLink,
            "content":main_content
        })
    return result

In [15]:
#   前綴與首頁網址
prefix = "https://www.ptt.cc"
url = "https://www.ptt.cc/bbs/HatePolitics/index.html"

#   請求與解析
res = requests.get(url)
soup = bs(res.text,"lxml")


#   CH4 - 建立資料集並匯出
#   首次呼叫 Def
output = []
result = get_ptt_post(soup)

#   將結果韓式返回的內容存入output陣列
output += result

#   再次呼叫數(N)頁 - 前頁網址
N = 1
previous_page = soup.select("div#action-bar-container div.action-bar div.btn-group-paging a")[1]["href"]
# Extract the page number using split and indexing
page_number = int(previous_page.split('/')[-1].split('.')[0].replace('index',''))

for i in range(N):
    url = "https://www.ptt.cc/bbs/HatePolitics/index{}".format(page_number-i)+".html"
    res = requests.get(url)
    soup = bs(res.text,"lxml")

    result = get_ptt_post(soup)
    output += result
    print("{} is ok".format(url))

#   產出資料集
df = pd.DataFrame(output)

#   欄位重新命名
df.rename(columns={
    "title":'標題',
    "date":'發文時間',
    "author":'作者',
    "link":'網址',
    "content":'內文'
    }, inplace=True)

https://www.ptt.cc/bbs/HatePolitics/index4164.html is ok


In [16]:

# 假设需要处理的文本在某一列中，例如 'content' 列
# 请确认您文件中实际的文本列名
if '內文' in df.columns:
    def process_text(text):
        # Check if the input is a string
        if isinstance(text, str):
            # 正则表达式处理文本
            pattern = r'標題(.*?)時間.*?\n(.*)'
            match = re.search(pattern, text, re.DOTALL)
            if match:
                title = match.group(1).strip()
                content = match.group(2).strip()
                return f'{title}\n{content}'
        return text  # 如果不匹配，返回原文本或者非字符串类型

    # 应用正则表达式处理每一行
    df['內文'] = df['內文'].apply(process_text)

In [17]:
clean_text = []
for text in df['內文']:
  # Check if the value is a string before applying re.sub
  if isinstance(text, str):
    clean = re.sub(r'(\n|\t|\r|[a-z]|[A-Z]|http\S+)', r'', text)
    clean_text.append(clean)
  else:
    # Handle non-string values (e.g., NaN)
    clean_text.append('')  # Or any other appropriate value

# 清理完的text存成新的一欄 data
df['內文'] = clean_text
df=df[['內文']]



In [18]:
output_file = 'clean_processed_ptt_tech_job_post.csv'
# 檢查 'clean_text' 欄位是否存在
if '內文' in df.columns:
    # 篩選資料，移除含有 "[新聞]" 的列
    filtered_df = df[~df['內文'].str.contains(r'\[新聞\]', na=False)]
    # 儲存到新的CSV檔案
    filtered_df.to_csv(output_file, index=False)

    print(f"已移除含有 '[新聞]' 的列，結果儲存至 {output_file}")

已移除含有 '[新聞]' 的列，結果儲存至 clean_processed_ptt_tech_job_post.csv
